# Link do repositório no Github:

https://github.com/NPaiffer/OdontoprevIOT?tab=readme-ov-file

In [ ]:
# Carregar o arquivo

from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv('/content/PS_20174392719_1491204439457_log.csv')
print(df.head())

: 

In [2]:
import os
print(os.listdir())

['.config', 'PS_20174392719_1491204439457_log.csv', 'sample_data']


Verificar os tipos de transação e a distribuição de fraudes

In [3]:
# Verificar o número de fraudes e transações normais

print(df['isFraud'].value_counts())

isFraud
0.0    409278
1.0       209
Name: count, dtype: int64


In [4]:
# Verificar os tipos de transações

print(df['type'].value_counts())

type
CASH_OUT    147466
PAYMENT     136704
CASH_IN      88583
TRANSFER     33801
DEBIT         2933
CASH_            1
Name: count, dtype: int64


In [5]:
# Verificar valores nulos

print(df.isnull().sum())

step              0
type              0
amount            1
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64


# Treinar o Modelo Isolation Forest

In [6]:
# Imports

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

In [7]:
# Selecionar as features para o modelo

X = df[['amount', 'oldbalanceOrg', 'newbalanceOrig']]
y = df['isFraud']

In [8]:
# Dividir os dados em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Criar o modelo Isolation Forest

model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
model.fit(X_train)

IsolationForest(contamination=0.05, random_state=42)

In [10]:
# Fazer previsões

y_pred = model.predict(X_test)
y_pred = [1 if x == -1 else 0 for x in y_pred]

# Avaliar o Modelo

In [20]:
# Imports

import numpy as np

print("Valores NaN em y_test:", np.isnan(y_test).sum())

Valores NaN em y_test: 1


In [21]:
# Contar quantos NaN existem em y_test

y_test_clean = y_test[~np.isnan(y_test)]
y_pred_clean = y_pred[:len(y_test_clean)]

In [27]:
# Rodamos a matriz de confusão

from sklearn.metrics import confusion_matrix, classification_report



In [28]:
# Avaliar o modelo

cm = confusion_matrix(y_test_clean, y_pred_clean)
print("Matriz de Confusão:")
print(cm)

report = classification_report(y_test_clean, y_pred_clean)
print("Relatório de Classificação:")
print(report)

Matriz de Confusão:
[[77815  4041]
 [   37     4]]
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       1.00      0.95      0.97     81856
         1.0       0.00      0.10      0.00        41

    accuracy                           0.95     81897
   macro avg       0.50      0.52      0.49     81897
weighted avg       1.00      0.95      0.97     81897



In [29]:
print("Tamanho y_test:", len(y_test))
print("Tamanho y_pred:", len(y_pred))
print("Valores únicos em y_pred:", set(y_pred))


Tamanho y_test: 81898
Tamanho y_pred: 81898
Valores únicos em y_pred: {0, 1}


# Escolhendo Algoritmos para Testar

In [30]:
# Imports

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
# Carregar os dados

df = pd.read_csv('/content/PS_20174392719_1491204439457_log.csv')

In [32]:
# Selecionar as features (removendo colunas não numéricas)

X = df[['amount', 'oldbalanceOrg', 'newbalanceOrig']]
y = df['isFraud']

In [33]:
# Normalizar os dados para melhorar o desempenho de alguns modelos

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [34]:
# Dividir entre treino e teste

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modelo 1: Árvores de Decisão

In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

print("Árvore de Decisão:")
print(classification_report(y_test, y_pred_dt))


Árvore de Decisão:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270904
           1       0.91      0.96      0.93      1620

    accuracy                           1.00   1272524
   macro avg       0.95      0.98      0.97   1272524
weighted avg       1.00      1.00      1.00   1272524



# Modelo 2: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest:")
print(classification_report(y_test, y_pred_rf))


# Modelo 3: XGBoost

In [ ]:
# Instalar o XGBoost

!pip install xgboost

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost:")
print(classification_report(y_test, y_pred_xgb))


# Modelo 4: Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print("Regressão Logística:")
print(classification_report(y_test, y_pred_lr))

# Modelo 5: SVM (Support Vector Machine)

In [ ]:
from sklearn.svm import SVC

svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

print("SVM:")
print(classification_report(y_test, y_pred_svm))

# Matriz de Confusão para cada Modelo

Matriz de confusão ajuda a entender alguns erros cometidos pelo modelo e mostra os verdadeiros positivos e falsos positivos, verdadeiros negativos e falsos negativos.

In [1]:
# Imports

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Função para desenhar matriz de confusão

def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Fraude"], yticklabels=["Normal", "Fraude"])
    plt.xlabel("Previsto")
    plt.ylabel("Real")
    plt.title(f"Matriz de Confusão - {model_name}")
    plt.show()

In [ ]:
# Gerar matriz para cada modelo

plot_confusion_matrix(y_test, y_pred_dt, "Árvore de Decisão")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest")
plot_confusion_matrix(y_test, y_pred_xgb, "XGBoost")
plot_confusion_matrix(y_test, y_pred_lr, "Regressão Logística")
plot_confusion_matrix(y_test, y_pred_svm, "SVM")

# Comparação das Métricas (Precisão, Recall, F1-Score)

Gráfico de barras para comparação das metricas dos modelos

In [ ]:
# Imports

import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Dicionário para armazenar as métricas

metrics = {
    "Modelo": ["Árvore de Decisão", "Random Forest", "XGBoost", "Regressão Logística", "SVM"],
    "Precisão": [
        precision_score(y_test, y_pred_dt),
        precision_score(y_test, y_pred_rf),
        precision_score(y_test, y_pred_xgb),
        precision_score(y_test, y_pred_lr),
        precision_score(y_test, y_pred_svm),
    ],
    "Recall": [
        recall_score(y_test, y_pred_dt),
        recall_score(y_test, y_pred_rf),
        recall_score(y_test, y_pred_xgb),
        recall_score(y_test, y_pred_lr),
        recall_score(y_test, y_pred_svm),
    ],
    "F1-Score": [
        f1_score(y_test, y_pred_dt),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_xgb),
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_svm),
    ]
}

df_metrics = pd.DataFrame(metrics)

In [ ]:
# Plotando o gráfico de comparação das métricas

df_metrics.set_index("Modelo").plot(kind="bar", figsize=(10, 6), colormap="viridis")
plt.title("Comparação de Modelos - Precisão, Recall e F1-Score")
plt.ylabel("Pontuação")
plt.xticks(rotation=30)
plt.legend(loc="lower right")
plt.show()

# Distribuição das Fraudes no Dataset

In [ ]:
sns.countplot(x=df["isFraud"], palette="pastel")
plt.title("Distribuição de Fraudes no Dataset")
plt.xlabel("Fraude (0 = Normal, 1 = Fraudulenta)")
plt.ylabel("Quantidade")
plt.show()